In [1]:
import numpy as np
import pandas as pd
import heapq as hp
import multiprocessing as mp
from time import sleep
from binance.client import Client

In [81]:
class StockMarket:
    # эта функция вызывается когда мы хотим купить amount монеток по текущей цене
    def make_buy_order(self, currency_pair, amount):
        amount *= (1 + self.fee)
        spent = 0
        res_amount = amount
        for idx in range(len(self.orders['asks'])):
            if self.balance[1] <= 0 or amount <= 0:
                break
            can_afford = self.balance[1] / self.orders['asks'][idx][0] # сколько можем купить исходя из баланса
            spent_on_cur_iter = min(self.orders['asks'][idx][1], amount, can_afford) * self.orders['asks'][idx][0]
            spent += spent_on_cur_iter
            self.balance[1] -= spent_on_cur_iter
            amount -= spent_on_cur_iter / self.orders['asks'][idx][0]
        if not self.spend_till_end and amount > 0:
            return [0, self.balance[1]]
        return [res_amount - amount, -spent]

    
    # если хотим продать amount монеток по рыночной цене - нам сюда
    def make_sell_order(self, currency_pair, amount):
        amount *= (1 - self.fee)
        if not self.spend_till_end and self.balance[0] < amount:
            return [0, self.balance[0]]
        capital = 0
        amount = min(amount, self.balance[0])
        res_amount = amount
        for idx in range(len(self.orders['bids'])):
            if amount <= 0:
                break
            can_afford = min(amount, self.orders['bids'][idx][1])
            amount -= can_afford
            capital += can_afford * self.orders['bids'][idx][0]
        return [-res_amount, capital]
               
        
    # на вход поступает словарик {валютная пара: сколько хотим купить(если число отрицательное - продать)}
    # словарик с балансом {валюта: количество, которым мы располагаем}, словарик-стакан по всем нужным валютным 
    # парам и флаг - что мы делаем, если у нас не хватает баланса для покупки желаемого количества валюты
    # по умолчанию мы покупаем столько, сколько позволяют наши средства
    def query(self, query_dict, balance, orders, spend_till_end=True): 
        self.spend_till_end = spend_till_end
        result = dict()
        for pair in query_dict:
            self.orders = orders[pair]
            self.balance = [balance[pair[:3]], balance[pair[3:]]]
            print(self.balance)
            if query_dict[pair] > 0:
                result[pair] = self.make_buy_order(pair, query_dict[pair])
            elif query_dict[pair] < 0:
                result[pair] = self.make_sell_order(pair, -query_dict[pair])
            balance[pair[:3]] += result[pair][0]
            balance[pair[3:]] += result[pair][1]
        return result # возвращаем словарик с изменениями в количестве валюты А и В
    
    
    def __init__ (self, ):
        self.bnb_fee = 0.00075
        self.fee = 0.001
        self.orders = []
        with open('min_order_size_and_step.txt', 'r') as f:
            self.min_order_size = eval(f.read())

In [82]:
market = StockMarket()

In [2]:
# это для тестирования, на самом деле словарик {ваоютная пара: {bids : [[price1, quantity1], ...],
# asks: [[...]]}, ...} должен предоставлять DataCather

def get_orderbook(): 
    api_key = ''
    api_secret = ''
    client = Client(api_key, api_secret)
    orders = {'btcusdt': client.get_order_book(symbol='BTCUSDT'),
                          'ltcbtc': client.get_order_book(symbol='LTCBTC')}
    for pair in orders:
        orders[pair]['bids'] = [[float(orders[pair]['bids'][i][0]),
                                 float(orders[pair]['bids'][i][1])]for i in range(len(orders[pair]['bids']))]
        orders[pair]['asks'] = [[float(orders[pair]['asks'][i][0]),
                                 float(orders[pair]['asks'][i][1])]for i in range(len(orders[pair]['asks']))]
    return orders

In [3]:
get_orderbook()

{'btcusdt': {'lastUpdateId': 2210861756,
  'bids': [[9860.35, 0.002882],
   [9860.32, 0.088564],
   [9860.3, 0.058554],
   [9860.29, 0.088564],
   [9860.26, 2.0],
   [9860.14, 0.5],
   [9860.13, 2.0],
   [9860.11, 0.2],
   [9860.1, 1.939856],
   [9860.06, 0.2],
   [9859.97, 0.083139],
   [9859.92, 0.002027],
   [9859.34, 0.07508],
   [9859.33, 1.006798],
   [9859.29, 1.36489],
   [9859.24, 0.202748],
   [9859.19, 0.00132],
   [9858.46, 0.060423],
   [9857.88, 0.3],
   [9857.53, 0.00516],
   [9856.92, 0.167291],
   [9856.89, 0.101398],
   [9856.67, 0.25],
   [9856.37, 0.606],
   [9856.3, 0.1151],
   [9856.13, 4.7],
   [9855.51, 0.121661],
   [9855.43, 0.0721],
   [9854.98, 0.2],
   [9854.97, 0.3],
   [9854.92, 0.340937],
   [9854.86, 0.01],
   [9854.29, 0.101064],
   [9854.0, 0.3],
   [9853.98, 0.05056],
   [9853.9, 0.005],
   [9853.5, 0.01758],
   [9853.44, 0.07],
   [9853.0, 0.43045],
   [9852.95, 1.962308],
   [9852.83, 0.0375],
   [9852.78, 0.452317],
   [9852.77, 0.010153],
   [985

In [84]:
market.query({'btcusdt': 3, 'ltcbtc': -5}, {'btc': 2, 'ltc': 3, 'usdt': 2e4}, get_orderbook())

[2, 20000.0]
[3, 4.034160491003993]


{'btcusdt': [2.034160491003993, -20000.0],
 'ltcbtc': [-3, 0.023918999999999996]}

In [52]:
min_order_size = {
 'btcusdt': [0.000001, 10, 0.01],
 'ethusdt': [0.00001, 10, 0.01],
 'bchusdt': [0.00001, 10, 0.01],
 'bnbusdt': [0.01, 10, 0.0001],
 'ltcusdt': [0.00001, 10, 0.01],
 'ethbtc': [0.001, 0.0001, 0.000001],
 'bchbtc': [0.001, 0.0001, 0.000001],
 'bnbbtc': [0.01, 0.0001, 0.0000001],
 'ltcbtc': [0.01, 0.0001, 0.000001],
 'bchbnb': [0.001, 0.1, 0.001],
 'ltcbnb': [0.001, 0.1, 0.001]
}

In [57]:

print(d)

{'btcusdt': [1e-06, 10, 0.01], 'ethusdt': [1e-05, 10, 0.01], 'bchusdt': [1e-05, 10, 0.01], 'bnbusdt': [0.01, 10, 0.0001], 'ltcusdt': [1e-05, 10, 0.01], 'ethbtc': [0.001, 0.0001, 1e-06], 'bchbtc': [0.001, 0.0001, 1e-06], 'bnbbtc': [0.01, 0.0001, 1e-07], 'ltcbtc': [0.01, 0.0001, 1e-06], 'bchbnb': [0.001, 0.1, 0.001], 'ltcbnb': [0.001, 0.1, 0.001]}
